In [1]:
from IPython.display import clear_output
import torch
import os
from IPython.display import Image, clear_output  # to display images
from hyperopt import fmin, tpe, hp, Trials, space_eval
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt


print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.0.1+cu117 (NVIDIA GeForce GTX 1660 Ti)


In [2]:
def on_train_epoch_end(predictor):
    clear_output()

In [3]:
def objective(params):
    # Define el modelo con los parámetros dados
    model = YOLO().load('yolov8n.pt')
    
    model.add_callback("on_train_epoch_end", on_train_epoch_end)
    model.add_callback("on_val_end", on_train_epoch_end)
    
    # Entrena el modelo
    model.train(data="/tf/data/Mammographies/yolo_data/data_object_detection.yaml",
                epochs=5,
                dropout=params["dropout"],
                lr0=params["lr"],                
                batch=1,
                amp=False,
                augment=True,
                verbose=False,
                imgsz=1600)
    
    # Evalúa el modelo
    metrics = model.val()
    
    # Devuelve la métrica a optimizar
    return -metrics.results_dict["metrics/recall(B)"]

# Define los rangos de los hiperparámetros
space = {
    'dropout': hp.choice('dropout',[0.0, 0.2, 0.4]),
    'lr': hp.loguniform('lr', -6, -1),
}

trials = Trials()

# Realiza la búsqueda de hiperparámetros
best = fmin(objective, space, algo=tpe.suggest, max_evals=10, trials=trials)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

Transferred 355/355 items from pretrained weights
New https://pypi.org/project/ultralytics/8.0.110 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.109 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 Ti, 5945MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/tf/data/Mammographies/yolo_data/data_object_detection.yaml, epochs=5, patience=50, batch=1, imgsz=1600, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width


        1/5       1.7G       2.89      19.75      2.592        148       1600:   0%|          | 9/2442 [00:02<09:09,  4.43it/s]

        1/5       1.7G       2.95      18.31      2.543         75       1600:   0%|          | 9/2442 [00:03<09:09,  4.43it/s]

        1/5       1.7G       2.95      18.31      2.543         75       1600:   0%|          | 10/2442 [00:03<08:51,  4.57it/s]

        1/5       1.7G      2.998      17.76       2.64          1       1600:   0%|          | 10/2442 [00:03<08:51,  4.57it/s]

        1/5       1.7G      2.998      17.76       2.64          1       1600:   0%|          | 11/2442 [00:03<08:38,  4.69it/s]

        1/5       1.7G      3.136      17.16      2.805          1       1600:   0%|          | 11/2442 [00:03<08:38,  4.69it/s]

        1/5       1.7G      3.136      17.16      2.805          1       1600:   0%|          | 12/2442 [00:03<08:17,  4.89it/s]

        1/5       1.7G      3.164      16.29      2.749         12       1600:   0%|       


        1/5       1.7G      2.865      9.236      2.759          2       1600:   2%|1         | 40/2442 [00:09<07:44,  5.17it/s]

        1/5       1.7G      2.881       9.14      2.799          9       1600:   2%|1         | 40/2442 [00:09<07:44,  5.17it/s]

        1/5       1.7G      2.881       9.14      2.799          9       1600:   2%|1         | 41/2442 [00:09<07:52,  5.08it/s]

        1/5       1.7G      2.887      9.054      2.802          2       1600:   2%|1         | 41/2442 [00:09<07:52,  5.08it/s]

        1/5       1.7G      2.887      9.054      2.802          2       1600:   2%|1         | 42/2442 [00:09<07:45,  5.16it/s]

        1/5       1.7G       2.82      8.888      2.737          0       1600:   2%|1         | 42/2442 [00:09<07:45,  5.16it/s]

        1/5       1.7G       2.82      8.888      2.737          0       1600:   2%|1         | 43/2442 [00:09<07:44,  5.16it/s]

        1/5       1.7G      2.839      8.793      2.742          1       1600:   2%|1    


        1/5      1.71G      2.668      7.155      2.555          3       1600:   3%|2         | 71/2442 [00:15<07:38,  5.17it/s]

        1/5      1.71G      2.695      7.141      2.577          1       1600:   3%|2         | 71/2442 [00:15<07:38,  5.17it/s]

        1/5      1.71G      2.695      7.141      2.577          1       1600:   3%|2         | 72/2442 [00:15<07:24,  5.33it/s]

        1/5      1.71G      2.694      7.117      2.576          3       1600:   3%|2         | 72/2442 [00:15<07:24,  5.33it/s]

        1/5      1.71G      2.694      7.117      2.576          3       1600:   3%|2         | 73/2442 [00:15<07:36,  5.19it/s]

        1/5      1.71G      2.658      7.029      2.541          0       1600:   3%|2         | 73/2442 [00:15<07:36,  5.19it/s]

        1/5      1.71G      2.658      7.029      2.541          0       1600:   3%|3         | 74/2442 [00:15<07:17,  5.41it/s]

        1/5      1.71G      2.657      7.007       2.54          1       1600:   3%|3    

...


        1/5      1.71G      2.242      4.199      2.052          1       1600:  76%|#######5  | 1855/2442 [05:48<01:49,  5.34it/s]

        1/5      1.71G      2.242      4.199      2.052        142       1600:  76%|#######5  | 1855/2442 [05:48<01:49,  5.34it/s]

        1/5      1.71G      2.242      4.199      2.052        142       1600:  76%|#######6  | 1856/2442 [05:48<01:54,  5.14it/s]

        1/5      1.71G      2.241      4.199      2.052          1       1600:  76%|#######6  | 1856/2442 [05:48<01:54,  5.14it/s]

        1/5      1.71G      2.241      4.199      2.052          1       1600:  76%|#######6  | 1857/2442 [05:48<01:52,  5.18it/s]

        1/5      1.71G      2.241      4.199      2.052          3       1600:  76%|#######6  | 1857/2442 [05:49<01:52,  5.18it/s]

        1/5      1.71G      2.241      4.199      2.052          3       1600:  76%|#######6  | 1858/2442 [05:49<01:49,  5.32it/s]

        1/5      1.71G      2.242      4.198      2.053          1       16


        1/5      1.71G      2.238      4.187      2.047          1       1600:  77%|#######7  | 1885/2442 [05:54<01:45,  5.29it/s]

        1/5      1.71G      2.238      4.187      2.047          1       1600:  77%|#######7  | 1886/2442 [05:54<01:43,  5.39it/s]

        1/5      1.71G      2.239      4.187      2.047          2       1600:  77%|#######7  | 1886/2442 [05:54<01:43,  5.39it/s]

        1/5      1.71G      2.239      4.187      2.047          2       1600:  77%|#######7  | 1887/2442 [05:54<01:44,  5.32it/s]

        1/5      1.71G      2.239      4.186      2.048          2       1600:  77%|#######7  | 1887/2442 [05:54<01:44,  5.32it/s]

        1/5      1.71G      2.239      4.186      2.048          2       1600:  77%|#######7  | 1888/2442 [05:54<01:44,  5.28it/s]

        1/5      1.71G      2.239      4.187      2.048          1       1600:  77%|#######7  | 1888/2442 [05:54<01:44,  5.28it/s]

        1/5      1.71G      2.239      4.187      2.048          1       16


        1/5      1.71G      2.236      4.175      2.046          1       1600:  78%|#######8  | 1916/2442 [05:59<01:41,  5.17it/s]

        1/5      1.71G      2.236      4.176      2.046          5       1600:  78%|#######8  | 1916/2442 [06:00<01:41,  5.17it/s]

        1/5      1.71G      2.236      4.176      2.046          5       1600:  79%|#######8  | 1917/2442 [06:00<01:42,  5.13it/s]

        1/5      1.71G      2.237      4.176      2.046          1       1600:  79%|#######8  | 1917/2442 [06:00<01:42,  5.13it/s]

        1/5      1.71G      2.237      4.176      2.046          1       1600:  79%|#######8  | 1918/2442 [06:00<01:50,  4.72it/s]

        1/5      1.71G      2.235      4.174      2.045          0       1600:  79%|#######8  | 1918/2442 [06:00<01:50,  4.72it/s]

        1/5      1.71G      2.235      4.174      2.045          0       1600:  79%|#######8  | 1919/2442 [06:00<01:44,  5.01it/s]

        1/5      1.71G      2.236      4.174      2.045          8       16


        1/5      1.71G      2.233      4.167      2.045          1       1600:  80%|#######9  | 1946/2442 [06:05<01:32,  5.36it/s]

        1/5      1.71G      2.233      4.167      2.045          1       1600:  80%|#######9  | 1947/2442 [06:05<01:31,  5.40it/s]

        1/5      1.71G      2.234      4.167      2.045          1       1600:  80%|#######9  | 1947/2442 [06:06<01:31,  5.40it/s]

        1/5      1.71G      2.234      4.167      2.045          1       1600:  80%|#######9  | 1948/2442 [06:06<01:30,  5.43it/s]

        1/5      1.71G      2.235      4.167      2.045          1       1600:  80%|#######9  | 1948/2442 [06:06<01:30,  5.43it/s]

        1/5      1.71G      2.235      4.167      2.045          1       1600:  80%|#######9  | 1949/2442 [06:06<01:33,  5.30it/s]

        1/5      1.71G      2.234      4.167      2.045          2       1600:  80%|#######9  | 1949/2442 [06:06<01:33,  5.30it/s]

        1/5      1.71G      2.234      4.167      2.045          2       16


        1/5      1.71G      2.238      4.164      2.045          1       1600:  81%|########  | 1977/2442 [06:11<01:28,  5.27it/s]

        1/5      1.71G      2.239      4.165      2.046          1       1600:  81%|########  | 1977/2442 [06:11<01:28,  5.27it/s]

        1/5      1.71G      2.239      4.165      2.046          1       1600:  81%|########  | 1978/2442 [06:11<01:26,  5.35it/s]

        1/5      1.71G      2.239      4.164      2.046          1       1600:  81%|########  | 1978/2442 [06:11<01:26,  5.35it/s]

        1/5      1.71G      2.239      4.164      2.046          1       1600:  81%|########1 | 1979/2442 [06:11<01:27,  5.27it/s]

        1/5      1.71G      2.239      4.164      2.046          1       1600:  81%|########1 | 1979/2442 [06:12<01:27,  5.27it/s]

        1/5      1.71G      2.239      4.164      2.046          1       1600:  81%|########1 | 1980/2442 [06:12<01:30,  5.12it/s]

        1/5      1.71G       2.24      4.164      2.046          2       16

In [ ]:
trials.trials

In [ ]:
best = space_eval(space, best)

In [ ]:
best

In [ ]:
# Define el modelo con los parámetros dados
model = YOLO().load('yolov8n.pt')

# Entrena el modelo
model.train(data="/tf/data/Mammographies/yolo_data/data_object_detection.yaml",
            epochs=100,
            patience=10,
            amp=False
            dropout=best["dropout"],
            lr0=best["lr"],                
            batch=1,
            augment=True,
            imgsz=1600)    

In [ ]:
# Evalúa el modelo
metrics = model.val()

In [ ]:
metrics

In [ ]:
model.export()

In [ ]:
from PIL import Image
import numpy as np
image=Image.open(f"/tf/data/Mammographies/yolo_data/object_detection/train/images/Calc-Test_P_01635_RIGHT_CC.png").convert("RGB")
results=model(image)[0] #Saved on runs/detect/predict
results.boxes.data.tolist()